<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/separatemodeling_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [ ]:
import pandas as pd
import numpy as np

from ddop.newsvendor import RandomForestWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV


from ddop.metrics import make_scorer
from ddop.metrics import average_costs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Loading data-set

*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## Split in feature and target matrix

In [ ]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split

In [ ]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## Definition of the item_ids 


*   will be needed within the for loops later
*   will contain all 25 products

In [ ]:
prods = list(range(0,25))

## Determining some parameters
*   the potential params and their values for randomized search are defined
*   the estimator, the scorer and finally the whole grid is defined

In [ ]:
max_depth = np.array([7, 9, 20, 50])
n_estimators = np.array([50, 70, 90, 100, 120])
min_samples_leaf = np.array([3, 4, 5])
param_random = dict(max_depth = max_depth, n_estimators = n_estimators, min_samples_leaf = min_samples_leaf)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# rf for 95% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,95
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_maxd_95 = []
best_param_nest_95 = []
best_param_minsl_95 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_95 = ((cu-(0.95*cu))/0.95)
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_95, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=4, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_95.append(maxd)
  best_param_nest_95.append(nest)
  best_param_minsl_95.append(minsl)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)


In [ ]:
result_rf_95_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_95 = ((cu-(0.95*cu))/0.95)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_95, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_95[prod], n_estimators = best_param_nest_95[prod],
                                       min_samples_leaf = best_param_minsl_95[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = rf.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_95),2)
  result_rf_95_avc.append(avc)

In [ ]:
RF_separate_avc_95 = pd.DataFrame(result_rf_95_avc, columns=['RF_avc_95'])
RF_separate_avc_95.index.name = 'item_id'

In [ ]:
result_rf_95_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_95 = ((cu-(0.95*cu))/0.95)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_95, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_95[prod], n_estimators = best_param_nest_95[prod],
                                       min_samples_leaf = best_param_minsl_95[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_95)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_95)
  result_rf_95_pscr.append(pscr)

In [ ]:
RF_separate_pscr_95 = pd.DataFrame(result_rf_95_pscr, columns=['RF_pscr_95'])
RF_separate_pscr_95.index.name = 'item_id'

# rf for 90% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,9
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_maxd_90 = []
best_param_nest_90 = []
best_param_minsl_90 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_90 = ((cu-(0.90*cu))/0.90)
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_90.append(maxd)
  best_param_nest_90.append(nest)
  best_param_minsl_90.append(minsl)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_rf_90_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_90 = ((cu-(0.90*cu))/0.90)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_90[prod], n_estimators = best_param_nest_90[prod],
                                       min_samples_leaf = best_param_minsl_90[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = rf.predict(X_test.loc[prod])
  
  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_90),2)
  result_rf_90_avc.append(avc)

In [ ]:
RF_separate_avc_90 = pd.DataFrame(result_rf_90_avc, columns=['RF_avc_90'])
RF_separate_avc_90.index.name = 'item_id'

In [ ]:
result_rf_90_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_90 = ((cu-(0.90*cu))/0.90)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_90[prod], n_estimators = best_param_nest_90[prod],
                                       min_samples_leaf = best_param_minsl_90[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_90)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_90)
  result_rf_90_pscr.append(pscr)

In [ ]:
RF_separate_pscr_90 = pd.DataFrame(result_rf_90_pscr, columns=['RF_pscr_90'])
RF_separate_pscr_90.index.name = 'item_id'

# rf for 75% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,75
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_maxd_75 = []
best_param_nest_75 = []
best_param_minsl_75 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_75 = ((cu-(0.75*cu))/0.75)
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_75.append(maxd)
  best_param_nest_75.append(nest)
  best_param_minsl_75.append(minsl)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_rf_75_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_75 = ((cu-(0.75*cu))/0.75)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_75[prod], n_estimators = best_param_nest_75[prod],
                                       min_samples_leaf = best_param_minsl_75[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = rf.predict(X_test.loc[prod])
  
  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_75),2)
  result_rf_75_avc.append(avc)

In [ ]:
RF_separate_avc_75 = pd.DataFrame(result_rf_75_avc, columns=['RF_avc_75'])
RF_separate_avc_75.index.name = 'item_id'

In [ ]:
result_rf_75_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_75 = ((cu-(0.75*cu))/0.75)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_75[prod], n_estimators = best_param_nest_75[prod],
                                       min_samples_leaf = best_param_minsl_75[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_75)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_75)
  result_rf_75_pscr.append(pscr)

In [ ]:
RF_separate_pscr_75 = pd.DataFrame(result_rf_75_pscr, columns=['RF_pscr_75'])
RF_separate_pscr_75.index.name = 'item_id'

# rf for 50% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,5
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_maxd_50 = []
best_param_nest_50 = []
best_param_minsl_50 = []

for prod in prods:
  
  cu = X_train.loc[prod]['sell_price'].mean()
  co_50 = ((cu-(0.50*cu))/0.50)
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_50.append(maxd)
  best_param_nest_50.append(nest)
  best_param_minsl_50.append(minsl)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [ ]:
result_rf_50_avc = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_50 = ((cu-(0.50*cu))/0.50)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_50[prod], n_estimators = best_param_nest_50[prod],
                                       min_samples_leaf = best_param_minsl_50[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = rf.predict(X_test.loc[prod])
  
  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_50),2)
  result_rf_50_avc.append(avc)

In [ ]:
RF_separate_avc_50 = pd.DataFrame(result_rf_50_avc, columns=['RF_avc_50'])
RF_separate_avc_50.index.name = 'item_id'

In [ ]:
result_rf_50_pscr = []
for prod in prods:
  cu = X_train.loc[prod]['sell_price'].mean()
  co_50 = ((cu-(0.50*cu))/0.50)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_50[prod], n_estimators = best_param_nest_50[prod],
                                       min_samples_leaf = best_param_minsl_50[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_50)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_50)
  result_rf_50_pscr.append(pscr)

In [ ]:
RF_separate_pscr_50 = pd.DataFrame(result_rf_50_pscr, columns=['RF_pscr_50'])
RF_separate_pscr_50.index.name = 'item_id'

# Merging

In [ ]:
RF_separated_avc = pd.concat([RF_separate_avc_95, RF_separate_avc_90, RF_separate_avc_75, RF_separate_avc_50], axis=1)
RF_separated_avc

,RF_avc_95,RF_avc_90,RF_avc_75,RF_avc_50
item_id,,,,
0,0.58,1.04,2.33,4.01
1,0.48,0.89,1.92,3.35
2,1.21,2.12,4.44,7.44
3,2.51,4.56,10.17,19.20
4,1.05,1.93,4.27,8.17
5,0.61,1.14,2.48,4.53
6,0.68,1.28,2.78,5.09
7,0.96,1.81,4.12,7.66
8,0.48,0.86,1.89,3.53


In [ ]:
RF_separated_pscr = pd.concat([RF_separate_pscr_95, RF_separate_pscr_90, RF_separate_pscr_75, RF_separate_pscr_50], axis=1)
RF_separated_pscr

,RF_pscr_95,RF_pscr_90,RF_pscr_75,RF_pscr_50
item_id,,,,
0,0.428485,0.391692,0.325187,0.309254
1,0.515553,0.523653,0.481219,0.433511
2,0.261248,0.215333,0.154172,0.142922
3,0.694329,0.677797,0.630826,0.540158
4,0.414564,0.355018,0.263669,0.217244
5,0.444215,0.408348,0.373841,0.247712
6,0.511927,0.469538,0.466339,0.417958
7,0.493121,0.425091,0.267675,0.184004
8,0.434749,0.395269,0.312476,0.256377


# Saving Files

In [ ]:
RF_separated_avc.to_csv('/content/drive/MyDrive/M5/RF_separated_avc.csv')

RF_separated_pscr.to_csv('/content/drive/MyDrive/M5/RF_separated_pscr.csv')